In [3]:
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 153.4 MB/s eta 0:00:00


In [5]:
!pip install pandas folium geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 164.8 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

In [7]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [9]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(url) 
spacex_df.head() 

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [10]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [11]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [14]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for index, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    site_lat = row['Lat']
    site_lon = row['Long']
    coordinate = [site_lat, site_lon] 

    folium.Circle(
        location=coordinate,
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site_name)).add_to(site_map) 

    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % site_name
        )
    ).add_to(site_map) 

site_map

In [24]:
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=5)

for index, row in spacex_df.iterrows():
    site = row['Launch Site']
    lat = row['Lat']
    lon = row['Long']
    outcome = row['class']
    
    color = 'green' if outcome == 1 else 'red'
    popup_text = f"{site} - {'Success' if outcome == 1 else 'Failure'}"

    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=color),
        popup=popup_text
    ).add_to(site_map)

site_map

In [22]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [23]:
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=5)
marker_cluster = MarkerCluster().add_to(site_map)

for index, row in spacex_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    outcome = row['class']
    site = row['Launch Site']
    
    color = 'green' if outcome == 1 else 'red'
    popup_text = f"{site} - {'Success' if outcome == 1 else 'Failure'}"
    
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=color),
        popup=folium.Popup(popup_text, parse_html=True)
    ).add_to(marker_cluster)

site_map

In [26]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [28]:
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=5)


marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    lat = record['Lat']
    lon = record['Long']
    color = record['marker_color']  
    site = record['Launch Site']
    outcome = 'Success' if record['class'] == 1 else 'Failure'
    popup_text = f"{site} - {outcome}"
   
    marker = folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_text, parse_html=True),
        icon=folium.Icon(color=color)
    )
    
    marker_cluster.add_child(marker)

site_map

In [29]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [30]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [31]:
launch_lat = 28.563197
launch_lon = -80.576820

coast_lat = 28.56367
coast_lon = -80.57163

distance_coast = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

distance_marker = folium.Marker(
    location=[coast_lat, coast_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_coast:.2f} KM</b></div>',
    )
)

distance_marker.add_to(site_map)

folium.Marker(
    location=[launch_lat, launch_lon],
    popup="Launch Site: CCAFS SLC-40",
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

site_map

In [32]:
coordinates = [
    [launch_lat, launch_lon],
    [coast_lat, coast_lon]
]

lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')

site_map.add_child(lines)

site_map

In [33]:
launch_lat = 28.563197
launch_lon = -80.576820

road_lat = 28.56485
road_lon = -80.571

distance_to_road = calculate_distance(launch_lat, launch_lon, road_lat, road_lon)

road_marker = folium.Marker(
    location=[road_lat, road_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:purple;"><b>{distance_to_road:.2f} KM</b></div>',
    )
)
site_map.add_child(road_marker)

road_line = folium.PolyLine(
    locations=[[launch_lat, launch_lon], [road_lat, road_lon]],
    color='purple',
    weight=2
)
site_map.add_child(road_line)

site_map

In [34]:
launch_lat = 34.632834
launch_lon = -120.610745

isla_vista_lat = 34.41123
isla_vista_lon = -119.85646

distance_to_isla_vista = calculate_distance(launch_lat, launch_lon, isla_vista_lat, isla_vista_lon)

isla_vista_marker = folium.Marker(
    location=[isla_vista_lat, isla_vista_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:darkblue;"><b>{distance_to_isla_vista:.2f} KM</b></div>',
    )
)
site_map.add_child(isla_vista_marker)

isla_vista_line = folium.PolyLine(
    locations=[[launch_lat, launch_lon], [isla_vista_lat, isla_vista_lon]],
    color='darkblue',
    weight=2
)
site_map.add_child(isla_vista_line)

site_map